# **Introduction**
----------------

### Le but de ce projet est de créer un modèle de classification supervisée, permettant, à partir d'un jeu de données d'une compagnie d'assurance de classer des données. 
Ces données prennent la forme de profils clients, ces profils comprennent de nombreuses informations comme par exemple l'âge, le genre ou le credit score de l'assuré. La donnée importante pour la création de notre modèle et la connaissance de si, oui ou non, l'assuré à fait une demande d'indemnisation, cela pourra permettre à cette compagnie d'assurance de prévoir à l'avance selon le profil de nouveau clients, si ils feront une nouvelle demande. 



## PARTIE 1 Importation et lecture des données
Ce premier bout de code permet de récupérer et lire les données depuis notre fichier CSV afin d'avoir une première analyse superficielle, ce faisant, nous pouvons vérifier la forme que prennent ces dites données. Ainsi nous pouvons préciser l'objectif du problème, il faut analyser les données pour prédire la demande d'indemnisation 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Importation des données
data = pd.read_csv('car_insurance.csv')

# Examen des premières lignes et des informations sur les colonnes
first_rows = data.head()
info = data.info()
description = data.describe()


print(missing_values)
